# 🏠 Ev Fiyat Tahmin Projesi - Veri Keşfi ve Analizi

Bu notebook, Kaggle House Prices veri seti üzerinde veri keşfi ve temel analiz adımlarını içerir.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', lambda x: f"{x:.3f}")

print("✅ Kütüphaneler yüklendi ve ayarlandı")


## 📥 Veri Yükleme


In [ ]:
train = pd.read_csv('datasets/housePrice/house_price_train.csv')
test = pd.read_csv('datasets/housePrice/house_price_test.csv')
print(f"Train şekli: {train.shape}")
print(f"Test şekli : {test.shape}")
train.head()


## 🔎 Hızlı Genel Bakış

Aşağıda sayısal sütunların temel istatistikleri ve eksik değer sayıları yer alır.


In [ ]:
train.describe().T


In [ ]:
train.isnull().sum().sort_values(ascending=False).head(15)
